In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

from glob import glob
from tqdm import tqdm
from scipy.io import loadmat
from PIL import Image

import keras 

import seaborn as sns
plt.style.use('seaborn')
sns.set(style="ticks", color_codes=True)

state = 1
np.random.seed(1)


DATA_PATH = '/home/jiehuan/PycharmProjects/Image_edge/Data/BSR500/BSDS500/data'
ROOT_PATH = '/home/jiehuan'
GROUND_PATH = DATA_PATH + '/groundTruth'
ORI_IMAGE_PATH = DATA_PATH + '/images'
OUT_IMAGE_PATH = '/home/jiehuan/PycharmProjects/Image_edge/image'
MODEL_PATH = '/home/jiehuan/PycharmProjects/Image_edge/model'
WEIGHT_PATH = '/home/jiehuan/PycharmProjects/Image_edge/weight'

POSTDATA_PATH = '/home/jiehuan/PycharmProjects/Image_edge/Data/BSR500/BSDS500/data/dataAgumentation'


def save_fig(fig_id, tight_layout=True):
    if not os.path.exists(OUT_IMAGE_PATH):
        os.makedirs(OUT_IMAGE_PATH)
    path = os.path.join(OUT_IMAGE_PATH, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)


In [2]:
i=0

for foldname in tqdm(os.listdir(ORI_IMAGE_PATH)):
    if foldname == 'val' or foldname == 'train':
        for img in sorted(glob(ORI_IMAGE_PATH + '/' + foldname + '/*')):
            data = cv2.imread(img)
            if data is not None:
                shape = np.asarray(data).shape
                (filepath,tempfilename) = os.path.split(img)
                (filename,extension) = os.path.splitext(tempfilename)

                if shape == (481, 321, 3): #height width
                    out0 = cv2.resize(data, (480, 320))
                    data = cv2.resize(data, (320, 480)) #width height
                    data = Image.fromarray(data)
                    data = np.asarray(data.rotate(90, expand=True))
                    newname0 = POSTDATA_PATH +"/train/"+filename+"S.npy"
                    np.save(newname0, out0) # extension + S
                else:
                    data = cv2.resize(data, (480, 320))
                
                
            
                if i<271:
                    orgdata = POSTDATA_PATH +"/train/"+filename+".npy"                
                    newname1 = POSTDATA_PATH +"/train/"+filename+"T.npy"      
                else:
                    orgdata = POSTDATA_PATH +"/val/"+filename+".npy"
                    newname1 = POSTDATA_PATH +"/val/"+filename+"T.npy"
                
                
                out1 = np.fliplr(data)
                np.save(orgdata, data)      # original Data
                np.save(newname1, out1)     # horizontal flip + T
            
            i +=1

